In [1]:
import ipympl
from ipywidgets import *

from pylab import *
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import h5py
import numpy as np
import numpy.matlib as ml
import numpy.linalg as la
import numpy.ma as ma

from central_ckv import central_ckv

In [2]:
# import matlab .mat file into python
# get the model in a matlab file, stored as -v7.3
# the model components must be saved as individual variables, 
# not as a struct.  See the matlab code structfields2vars
mf='temp3.mat';

f = h5py.File(mf, "r")

In [3]:
c=np.squeeze(f['c'][:])
k=np.squeeze(f['k'][:])
Nd=np.squeeze(f['n_d'][:])
P=np.transpose(f['P'][:])
R=np.transpose(f['R'][:])
weights=np.squeeze(f['weights'][:])
index=np.squeeze(f['index'][:])
xtest=np.array([ 66.8, 1.06, 7.5, 38, 5.23, 5.25])
xtest=np.squeeze(np.transpose(xtest))

rmw=xtest[0]
H_b=xtest[1]
TS=xtest[2]
Vmax=xtest[3]
LonNorth=xtest[4]
LonSouth=xtest[5]
alpha = 0.55

p1min=np.min(P[:,0])*1.1; p1max=np.max(P[:,0])*0.9   # rmw
p2min=np.min(P[:,1])*1.1; p2max=np.max(P[:,1])*0.9   # H_b
p3min=np.min(P[:,2])*1.1; p3max=np.max(P[:,2])*0.9   # TS
p4min=np.min(P[:,3])*1.1; p4max=np.max(P[:,3])*0.9   # Vmax
p5min=np.min(P[:,4])*1.1; p5max=np.max(P[:,4])*0.9   # LonNorth
p6min=np.min(P[:,5])*1.1; p6max=np.max(P[:,5])*0.9   # LonSouth

dp1=(p1max-p1min)/20
dp2=(p2max-p2min)/20
dp3=(p3max-p3min)/20
dp4=(p4max-p4min)/20
dp5=(p5max-p5min)/20
dp6=(p6max-p6min)/20

# get the FEM grid parts from f to create a triangulation object
lon = np.squeeze(f['x'][:])
lat = np.squeeze(f['y'][:])
latmin = np.mean(lat)  # needed for scaling lon/lat plots
nv  = np.squeeze(f['e'][:,:] -1)
nv=np.transpose(nv)
triangulation = Tri.Triangulation(lon,lat, triangles=nv)

NodeIndices=np.squeeze(f['NodeIndices'][:])
NodeIndices=(NodeIndices-1).astype(int)

In [4]:
lon_offset=-82

xtest=np.array([rmw,H_b,TS,Vmax,LonNorth,LonSouth])
temp=central_ckv(P,R,c,k,weights,Nd,index,xtest)
vmin=0
vmax=5
levels = linspace(vmin,vmax,11)

# put response into variable sized as lon.shape
zhat=ma.array(np.zeros(triangulation.x.shape))
zhat[:]=zhat.fill_value
zhat[NodeIndices]=temp
zhat[zhat<0]=0

fig = plt.figure("RSM Test", figsize=(5,3), dpi=144);
ax = fig.add_axes([0.0, 0.1, 0.8, 0.8]) 
ax.set_aspect(1.0/np.cos(latmin * np.pi / 180.0))
m = Basemap(projection='cyl', llcrnrlon=-80.0, llcrnrlat=33.0, urcrnrlon=-74.0, urcrnrlat=37.0,
            lat_0=35.0, lon_0=-77.0, resolution='i')

contour = m.contourf(lon, lat, zhat, triangles=nv, ax=ax,tri=True, levels=levels, shading='faceted', alpha=alpha)
m.drawcoastlines()
m.drawstates()
m.drawrivers()
m.warpimage()

ax.grid(True)
ax.set_xlim((-80,-74))
ax.set_ylim((33,37))
ax.tick_params(axis='both', which='major', labelsize=8)
ax.plot([-80, -70],[33.5,33.5],'g-')
ax.plot([-80, -70],[36.0,36.0],'g-')
artists_LonSouth_point = ax.plot([LonSouth+lon_offset],[33.5],'r*-')
artists_LonNorth_point = ax.plot([LonNorth+lon_offset],[36.0],'r*-')
artists_path_line = ax.plot([LonNorth+lon_offset, LonSouth+lon_offset],[36.0,33.5],'y-')

# add colorbar
cbax = fig.add_axes([0.75, 0.1, 0.05, 0.8]) 
cb = plt.colorbar(contour, cax=cbax,  orientation='vertical')
cb.set_label('[m MSL]', fontsize=8)
cb.ax.tick_params(axis='both', which='major', labelsize=8)

fig_canvas = plt.gcf().canvas

In [5]:
rmw_slider      = FloatSlider(min=p1min,max=p1max,step=dp1,value=rmw)
H_b_slider      = FloatSlider(min=p2min,max=p2max,step=dp2,value=H_b)
TS_slider       = FloatSlider(min=p3min,max=p3max,step=dp3,value=TS)
Vmax_slider     = FloatSlider(min=p4min,max=p4max,step=dp4,value=Vmax)
LonNorth_slider = FloatSlider(min=p5min,max=p5max,step=dp5,value=LonNorth)
LonSouth_slider = FloatSlider(min=p6min,max=p6max,step=dp6,value=LonSouth)
alpha_slider    = FloatSlider(min=0.0, max=1.0, step=0.05, value=alpha)

In [6]:
def update_contour():
    global contour, xtest, temp, zhat

    xtest=np.array([rmw,H_b,TS,Vmax,LonNorth,LonSouth])
    temp=central_ckv(P,R,c,k,weights,Nd,index,xtest)

    # put response into variable sized as lon.shape
    zhat=ma.array(np.zeros(triangulation.x.shape))
    zhat[:]=zhat.fill_value
    zhat[NodeIndices]=temp
    zhat[zhat<0]=0

    for thing in contour.collections:
        thing.remove()
    contour = m.contourf(lon, lat, zhat, triangles=nv, ax=ax,tri=True, levels=levels, shading='faceted', alpha=alpha)

In [7]:
def update_rmw(change):
    global rmw
    rmw = change['new']
    update_contour()
    fig_canvas.draw_idle()

rmw_slider.observe(update_rmw, names=["value"])

In [8]:
def update_H_b(change):
    global H_b
    H_b = change['new']
    update_contour()
    fig_canvas.draw_idle()

H_b_slider.observe(update_H_b, names=["value"])

In [9]:
def update_TS(change):
    global TS
    TS = change['new']
    update_contour()
    fig_canvas.draw_idle()

TS_slider.observe(update_TS, names=["value"])

In [10]:
def update_Vmax(change):
    global Vmax
    Vmax = change['new']
    update_contour()
    fig_canvas.draw_idle()

Vmax_slider.observe(update_Vmax, names=["value"])

In [11]:
def update_LonNorth(change):
    global LonNorth, artists_LonNorth_point, artists_path_line
    LonNorth = change['new']
    artists_LonNorth_point.pop(0).remove()
    artists_LonNorth_point = ax.plot([LonNorth+lon_offset],[36.0],'r*-')
    artists_path_line.pop(0).remove()
    artists_path_line = ax.plot([LonNorth+lon_offset, LonSouth+lon_offset],[36.0,33.5],'y-')
    update_contour()
    fig_canvas.draw_idle()

LonNorth_slider.observe(update_LonNorth, names=["value"])

In [12]:
def update_LonSouth(change):
    global LonSouth, artists_LonSouth_point, artists_path_line
    LonSouth = change['new']
    artists_LonSouth_point.pop(0).remove()
    artists_LonSouth_point = ax.plot([LonSouth+lon_offset],[33.5],'r*-')
    artists_path_line.pop(0).remove()
    artists_path_line = ax.plot([LonNorth+lon_offset, LonSouth+lon_offset],[36.0,33.5],'y-')
    update_contour()
    fig_canvas.draw_idle()

LonSouth_slider.observe(update_LonSouth, names=["value"])

In [13]:
def update_alpha(change):
    global alpha, contour
    alpha = change['new']
    for thing in contour.collections:
        thing.remove()
    contour = m.contourf(lon, lat, zhat, triangles=nv, ax=ax,tri=True, levels=levels, shading='faceted', alpha=alpha)
    fig_canvas.draw_idle()

alpha_slider.observe(update_alpha, names=["value"])

In [14]:
slider_box = VBox([rmw_slider,H_b_slider,TS_slider,Vmax_slider,LonNorth_slider,LonSouth_slider,alpha_slider])
HBox([fig_canvas, slider_box])